In [30]:
# librairies loading

import os
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.llms import Ollama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


In [14]:
# data loading
current_path = os.getcwd()
path = os.path.join(current_path, '..', 'data', 'raw_data', 'cga-canal-canalsat-web.pdf')

loader = PyPDFLoader(path)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000,
                                               chunk_overlap = 200,
                                               separators = ["\n\n", "\n", " ", ""])

chunks =text_splitter.split_documents(docs)

print(f"Number of chunks created :{len(chunks)}")

Number of chunks created :770


In [22]:
# embeddings

model_name = "BAAI/bge-small-en-v1.5"
model_kwargs = {'device' : 'cpu'}
encode_kwargs = {'normalize_embeddings' : True}

embeddings = HuggingFaceBgeEmbeddings(
                    model_name = model_name,
                    model_kwargs = model_kwargs,
                    encode_kwargs = encode_kwargs)

path_chroma_db = os.path.join(current_path, '..', 'data', 'chroma_db')
chroma_db = Chroma.from_documents(documents = chunks,
                                  embedding = embeddings,
                                  persist_directory = path_chroma_db)

chroma_db.persist() # for reproductibility

C:\Users\mayen\AppData\Local\Temp\ipykernel_101212\940450228.py:17: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  chroma_db.persist() # for reproductibility


In [27]:
# modelisation
llm = Ollama(model = "llama3")

chorma_db_reloaded = Chroma(persist_directory = path_chroma_db, embedding_function = embeddings)
retriever = chorma_db_reloaded.as_retriever(search_kwargs= {"k" : 3}) # fetch the 3 best chunks?

C:\Users\mayen\AppData\Local\Temp\ipykernel_101212\791873308.py:2: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the `langchain-ollama package and should be used instead. To use it run `pip install -U `langchain-ollama` and import as `from `langchain_ollama import OllamaLLM``.
  llm = Ollama(model = "llama3")
C:\Users\mayen\AppData\Local\Temp\ipykernel_101212\791873308.py:4: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the `langchain-chroma package and should be used instead. To use it run `pip install -U `langchain-chroma` and import as `from `langchain_chroma import Chroma``.
  chorma_db_reloaded = Chroma(persist_directory = path_chroma_db, embedding_function = embeddings)


In [28]:
template = """
Tu es un expert en politiques d'entreprise, ton rôle est de répondre aux questions des clients
et visiteurs en utilisant UNIQUEMENT le contexte fourni ci-dessous.
Si tu ne peux pas trouver la réponse dans le contexte, dis clairement que tu ne sais pas.
Fournis une réponse complète et précise.

CONTRXT:
{context}

QUESTION:
{question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [33]:
def format_docs(docs) :
    """
    format fetched context
    """
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
        {"context" : retriever | format_docs, "question" : RunnablePassthrough()}
        | prompt
        | llm
        |StrOutputParser()
    )

In [37]:
# test : 

question = "Quelles sont les conditions de resiliation des abonnements pour les offres de 26 ans?"

print(f"\n--- Question : {question} ---\n")
response = rag_chain.invoke(question)

print("--- Réponse du Chatbot ---\n")
print(response)


--- Question : Quelles sont les conditions de resiliation des abonnements pour les offres de 26 ans? ---

--- Réponse du Chatbot ---

Je ne vois pas d'information spécifique sur les offres de 26 ans dans le contexte fourni. Les conditions générales d'abonnement mentionnent les offres de 12 ou 24 mois, mais pas d'offre de 26 ans. Il est donc impossible pour moi de fournir des informations précises sur les conditions de résiliation pour ces offres.

Si vous avez des questions sur les conditions de résiliation pour les offres de 12 ou 24 mois, je serais ravi de vous aider.
